# MCF LP Problem Formulation

## Basic definitions

**Definition**: MCF Net

We define a **net** to be a tuple $(V, E, c, P, S, T, \sigma, d)$ where
    
 - $V$ is a non-empty set of vertices.
 - $G \subseteq V \times V - \{ (v, v) \mid v \in V \}$ is a set of (oriented) edges (self-loops are not included).
 - $c$ is a function $c : E \to \mathbb{R}_{0}^{+}$ which assigns a *capacity* to each edge.
 - $P$ is a non-empty set of *products*.
 - $S, T \subseteq V$ are in order a set of *source* and *destination* nodes
 - $\sigma$ is a function $\sigma : S \times P \to \mathbb{R}^{+}$ which assigns each supply node the amount of the given product available.
 - $d$ is a function $\delta : T \times P \to \mathbb{R}^{+}$ which assigns each demand node a demand for the given product.
 
 For simplicity we denote $\sigma_s^k := \sigma(s, k)$ and $d_t^k := d(s, k)$.
 
**Definition**: Neighbourhood

For oriented graph $G = (V, E)$ we define for each $v \in V$
$$
\begin{gather*}
N^{+}(v) = \{ u \mid (u, v) \in E \} \\
N^{-}(v) = \{ u \mid (v, u) \in E \}
\end{gather*}
$$
meaning $N^{+}$ is a set of all nodes with an edge comming into $v$ and $N^{-}$ is a set of all nodes for which $v$ has an outcomming edge.

**Definition**: Flow

Let $N = (V, E, c, P, S, T, \sigma, d)$ be a net. Then we define **flow** as a function
$$f : E \times P \to \mathbb{R}^{+}$$
satisfying the following conditions:
 - *the flow does not exceed edge cappacities*
     $$\forall e \in E : \sum\limits_{i \in P} f(e, i) \le c(e)$$
 - *the flow satisfies Kirchhoff's law for each procut and for all but source and destination nodes*
     $$\forall i \in P : \forall v \in (V - (S \cup T)) : \sum\limits_{u \in N^{+}(v)} f(uv, i) - \sum\limits_{u \in N^{-}(v)} f(vu, i) = 0$$
     
For simplicity we denote $f_e^k := f(e, k)$.

## LP Formulation

We are interested in a problem where demands of the destination nodes might not be met. We allow this and measure it with slack variables $\xi$. We will want to minimize these values and transport as much of the products as we can.

In the LP formulation we will have the following **variables**:

 - $f_e^k \in \mathbb{Q}^{+}, f_e^k \ge 0$ $\sim$ representing the flow of the product $k$ through the edge $e$
 - $\xi_v^k \in \mathbb{Q}^{+}, \xi_v^k \ge 0$ $\sim$ the slack variable for node $v \in T$ and product $k \in P$ (how much of the product are we still missing)
    
**Objective function**
$$ \min \sum_{(v, k) \in T \times P} \xi_v^k$$

**Given the conditions**
    
 - *respecting edge capacities*
 $$\forall e \in E : \sum\limits_{k \in P} f_e^k \le c(e)$$
 - *Kirchhoff's law*
 $$\forall v \in (V - (S \cup T)) : \forall k \in P : \sum\limits_{u \in N^{+}(v)} f_{uv}^k - \sum\limits_{u \in N^{-}(v)} f_{vu}^k = 0$$
 - *demands are satisfied given some slack*
 $$\forall t \in T : \forall k \in P : \sum_{u \in N^{+}(t)} f_{ut}^k + \xi_t^k = d_t^k$$
 - *supplies are not exhausted*
  $$\forall s \in S : \forall k \in P : \sum_{u \in N^{-}(s)} f_{su}^k \le \sigma_t^k$$
  
Adding additional variables $\varepsilon_e \ge 0$ for every edge in capacity conditions and $\varphi_s^k$ for every $s \in S$ and $k \in P$.

We arrive at the final LP formulation

**Objective function**
$$ \max \sum_{(v, k) \in T \times P} -\xi_v^k$$

**Conditions**
$$
\begin{align*}
\forall e \in E : &\sum\limits_{k \in P} f_e^k + \varepsilon_e = c(e) \\
\forall v \in (V - (S \cup T)) : \forall k \in P : &\sum\limits_{u \in N^{+}(v)} f_{uv}^k - \sum\limits_{u \in N^{-}(v)} f_{vu}^k = 0 \\
\forall t \in T : \forall k \in P : &\sum_{u \in N^{+}(t)} f_{ut}^k + \xi_t^k = d_t^k \\
\forall s \in S : \forall k \in P : &\sum_{u \in N^{-}(s)} f_{su}^k + \varphi_{s}^k = \sigma_t^k
\end{align*}
$$

**Variables**
$$
\begin{align*}
\forall e \in E : \forall k \in P &: f_e^k \in \mathbb{Q}^{+} && f_e^k \ge 0 \\
\forall t \in T : \forall k \in P &: \xi_t^k \in \mathbb{Q}^{+} && \xi_k \ge 0 \\
\forall e \in E &: \varepsilon_e \in \mathbb{Q}^{+} && \varepsilon_e \ge 0 \\
\forall s \in S : \forall k \in P &: \varphi_{s}^k \in \mathbb{Q}^{+} && \varphi_{s}^k \ge 0
\end{align*}
$$

Denoting $n$ the number of nodes and $m$ the number of edges, we have together $m + |P|n$ conditions and $(|P| + 1)m + (|S| + |D|)|P|$ variables.

### Collecting relevant information from instances

In [ ]:
import numpy as np
import sympy as sp
from pathlib import Path
from mcf_simplex_analyzer.load_instance import load_instance
from collections import defaultdict
from fractions import Fraction

from pprint import pprint

# Path to data directory
instances_path = Path("example/")

assert instances_path.exists()

instance_format = "planar"
nod_file = instances_path / (instance_format + ".nod")
arc_file = instances_path / (instance_format + ".arc")
sup_file = instances_path / (instance_format + ".sup")
mut_file = instances_path / (instance_format + ".mut")

instance = load_instance(instance_format, nod_file, arc_file, sup_file, mut_file)
print(instance.info)

In [ ]:
# Collect edge capacities
capacities = {}
commodities = set()

for arc in instance.arcs:
    fromnode, tonode, commodity, cost, individual_capacity, mutual_ptr = arc
    if fromnode == -1 or tonode == -1:
        print(arc)
    
    if commodity != -1:
        commodities.add(commodity)
        
    mutual, capacity = capacities.get((fromnode, tonode), (None, None))
    if mutual_ptr > 0:
        mutual = instance.mutual.capacity[mutual_ptr - 1]
    
    if individual_capacity >= 0:
        capacity = capacity + individual_capacity if capacity is not None else individual_capacity
    
    capacities[(fromnode, tonode)] = (mutual, capacity)

# Account for mutual cappacity
for key in capacities:
    mutual, total = capacities[key]
    capacities[key] = max(mutual, total) if mutual is not None and total is not None else total

print(capacities)
print(len(capacities))
print(commodities)

In [ ]:
# Collect source and destination vertices

source = {}
destination = {}

for supply in instance.supply:
    s, t, k, f = supply
    if k < 0:
        print(supply)
        
    if s == -1:
        destination.setdefault(t, dict())[k] = f
        
    if t == -1:
        source.setdefault(s, dict())[k] = f
        
print(source)
print(destination)

In [ ]:
in_neighbours = {}
out_neighbours = {}

for arc in instance.arcs:
    fromnode, tonode, _, _, _, _ = arc
    in_neighbours.setdefault(tonode, set()).add(fromnode)
    out_neighbours.setdefault(fromnode, set()).add(tonode)

    
print(in_neighbours)
print(out_neighbours)

# Finding maximal flow for individual commodities

In [ ]:
DRAW_GRAPH = False

In [ ]:
# Normalize to whole numbers
denoms = ([capacity.denominator for capacity in capacities.values() if capacity is not None] 
          + [ source[s][k].denominator for s in source for k in source[s] ] 
          + [ destination[t][k].denominator for t in destination for k in destination[t] ] 
         )
lcm = np.lcm.reduce(denoms)

# Normalize capacities
for key in capacities:
    capacity = capacities[key]
    if capacity is not None:
        capacities[key] = capacity.numerator * (lcm // capacity.denominator)
        
# Normalize sources
for s in source:
    for k in source[s]:
        supply = source[s][k]
        source[s][k] = supply.numerator * (lcm // supply.denominator)

# Normalize destinations
for t in destination:
    for k in destination[t]:
        demand = destination[t][k]
        destination[t][k] = demand.numerator * (lcm // demand.denominator)
        
network_info = (source, destination, capacities, lcm)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

graph = nx.DiGraph()
for (u, v) in capacities:
    capacity = capacities[(u, v)]
    if capacity is not None:
        graph.add_edge(u, v, capacity=capacity)
    else:
        graph.add_edge(u, v)
        
if DRAW_GRAPH:
    plt.figure(figsize=(13, 8))
    nx.draw_kamada_kawai(graph, node_size=100)

In [ ]:
import pprint

def gather_sources(network_info, graph, commodity):
    (source, _, capacities, _) = network_info

    for s in source:
        if commodity in source[s]:
            supply = source[s][commodity].numerator
            graph.add_edge("Source", s, capacity=supply)

            
def gather_destinations(network_info, graph, commodity):
    (_, destination, capacities, _) = network_info
    
    commodity_destinations = []
    for t in destination:
        if commodity in destination[t]: 
            demand = destination[t][commodity]      
            commodity_graph.add_edge(t, "Destination", capacity=demand)
            
def get_percantage_full(graph, flow_dict, u, v):
    max_capacity = graph.get_edge_data(u, v).get('capacity', None)
    
    if max_capacity is None:
        return 0
    
    perc = flow_dict[u][v] / max_capacity
    
    return perc

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize

def draw_network(graph, flow_dict):
    plt.figure(figsize=(10, 7))
    
    percentage_full=np.array([ get_percantage_full(graph, flow_dict, u, v) for (u,v) in graph.edges])
    norm = Normalize(vmin=0, vmax=1)
    colors = cm.viridis(norm(percentage_full))
    colors[ percentage_full > 1 ] = [1, 0, 0, 1]
    
    
    # DEBUG
    #print(percentage_full)
    
    nx.draw_kamada_kawai(
        graph,
        
        with_labels=True,
        font_size=16,
        font_weight="bold",
        
        node_size=500,
        nodelist=graph.nodes,
        node_color=['lightblue' for _ in graph.nodes],
        
        #edge_color=percentage_full,
        edge_color=colors,
        width=2,
        connectionstyle="arc3,rad=0.1"
    )

MAX_ITER = 4
print(f"Computing first {MAX_ITER} networks ...")

for commodity in range(1, min(instance.info.products_no + 1, MAX_ITER + 1)):
    commodity_graph = graph.copy()
    
    gather_sources(network_info, commodity_graph, commodity)
    gather_destinations(network_info, commodity_graph, commodity)

    # Compute maximal flow     
    flow_value, flow_dict = nx.maximum_flow(commodity_graph, "Source", "Destination")
    
    # DEBUG
    # print(flow_value)
    # print(pprint.pprint(flow_dict))
    
    if DRAW_GRAPH:
        draw_network(commodity_graph, flow_dict)

In [ ]:
flow_sum = {}

for commodity in range(1, instance.info.products_no + 1):
    print("Computing flow for commodity", commodity)
    
    commodity_graph = graph.copy()
    
    gather_sources(network_info, commodity_graph, commodity)
    gather_destinations(network_info, commodity_graph, commodity)

    # Compute maximal flow     
    flow_value, flow_dict = nx.maximum_flow(commodity_graph, "Source", "Destination")
    
    for u in flow_dict:
        if u == "Source":
            continue
            
        for v in flow_dict[u]:
            if v == "Destination":
                continue
                
            d = flow_sum.setdefault(u, {})
            d[v] = d.get(v, 0) + flow_dict[u][v]
            
    # DEBUG
    # print(flow_value)
    # print(pprint.pprint(flow_dict))

flow_sum

In [ ]:
draw_network(graph, flow_sum)